In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy import sparse
import anndata
import scanpy as sc
import scvelo as scv
scv.set_figure_params()
%config Completer.use_jedi = False
import seaborn as sb
import gc
import scrublet as scr

# Read Alignment with StarSolo

This scanpy code utilizes a .csv document as well as the output files of StarSolo.   
We provide the aligned folders (STAR output) which are imported later during this script as downloadable .tar archives:   
If the downloads do not function properly, please write to david.glaser@bio.uni-giessen.de or marek.bartkuhn@gen.bio.uni-giessen.de   

S3 storage:
https://s3.computational.bio.uni-giessen.de/swift/v1/plet1_paper_Pervizaj_2023/   

A1_wpre_outSolo.out.tar   
A2_wpre_outSolo.out.tar   
A3_wpre_outSolo.out.tar   
B1_wpre_outSolo.out.tar   
B2_wpre_outSolo.out.tar   
B3_wpre_outSolo.out.tar   
C1_new_wpre_outSolo.out.tar   
C1_wpre_outSolo.out.tar   
C2_new_wpre_outSolo.out.tar   
C2_wpre_outSolo.out.tar   
C3_new_wpre_outSolo.out.tar   
D1_wpre_outSolo.out.tar   
D2_wpre_outSolo.out.tar   
D3_wpre_outSolo.out.tar   
E1_wpre_outSolo.out.tar   
E2_wpre_outSolo.out.tar   
E3_wpre_outSolo.out.tar   

However, if you prefer aligning the data yourself, we used the following Star call (fastq files are available at GEO ):   

STAR   
--genomeDir /path/to/genome/index/   
--readFilese /path/to/XXX_R2_001.fastq.gz /path/to/XXX_R1_001.fastq.gz    
--soloType CB_UMI_Simple   
--runThreadN 8     
--soloCBwhitelist /path/to/10xBarcodes/3M-february-2018.txt   
--readFilesCommand zcat   
--outSAMtype BAM SortedByCoordinate   
--soloFeatures Gene Velocyto   
--clipAdapterType CellRanger4   
--outFilterScoreMin 30   
--soloCBmatchWLtype 1MM_multi_Nbase_pseudocounts   
--soloUMIfiltering MultiGeneUMI_CR   
--soloUMIdedup 1MM_CR   
--soloBarcodeReadLength 28 --soloCBstart 1 --soloCBlen 16 --soloUMIstart17 --soloUMIlen 12 --soloCellFilter EmptyDrops_CR   
--outFileNamePrefix /work/C1_new_wpre_out   

# Misc

In [ ]:
## Change to directory which contains "scanpyInput.csv" and Star output files (eg. A1_wpre_outSolo.out)

expName = "Infection_timecourse" # Name given to the experimental setup
os.chdir("/root/host_home/") # This is the path to files in the docker container.
os.getcwd()

# Input data

In [ ]:
df = pd.read_csv("scanpyInput.csv",sep=",")
df = df.sort_values(["timepoint","replicate"])
df

In [ ]:
## Function to automatically import Star output and transfer to anndata object

def buildAnndataFromStarCurr(path):
    """Generate an anndata object from the STAR aligner output folder"""
    path=path
    print(path)
    # Load Read Counts
    X = sc.read_mtx(path+'Gene/raw/matrix.mtx')

    # Transpose counts matrix to have Cells as rows and Genes as cols as expected by AnnData objects
    X = X.X.transpose()

    # Load the 3 matrices containing Spliced, Unspliced and Ambigous reads
    mtxU = np.loadtxt(path+'Velocyto/raw/unspliced.mtx', skiprows=3, delimiter=' ')
    mtxS = np.loadtxt(path+'Velocyto/raw/spliced.mtx', skiprows=3, delimiter=' ')
    mtxA = np.loadtxt(path+'Velocyto/raw/ambiguous.mtx', skiprows=3, delimiter=' ')

    # Extract sparse matrix shape informations from the third row
    shapeU = np.loadtxt(path+'Velocyto/raw/unspliced.mtx', skiprows=2, max_rows = 1 ,delimiter=' ')[0:2].astype(int)
    shapeS = np.loadtxt(path+'Velocyto/raw/spliced.mtx', skiprows=2, max_rows = 1 ,delimiter=' ')[0:2].astype(int)
    shapeA = np.loadtxt(path+'Velocyto/raw/ambiguous.mtx', skiprows=2, max_rows = 1 ,delimiter=' ')[0:2].astype(int)

    # Read the sparse matrix with csr_matrix((data, (row_ind, col_ind)), shape=(M, N))
    # Subract -1 to rows and cols index because csr_matrix expects a 0 based index
    # Traspose counts matrix to have Cells as rows and Genes as cols as expected by AnnData objects

    spliced = sparse.csr_matrix((mtxS[:,2], (mtxS[:,0]-1, mtxS[:,1]-1)), shape = shapeS).transpose()
    unspliced = sparse.csr_matrix((mtxU[:,2], (mtxU[:,0]-1, mtxU[:,1]-1)), shape = shapeU).transpose()
    ambiguous = sparse.csr_matrix((mtxA[:,2], (mtxA[:,0]-1, mtxA[:,1]-1)), shape = shapeA).transpose()

    # Load Genes and Cells identifiers
    obs = pd.read_csv(path+'Velocyto/raw/barcodes.tsv',
                  header = None, index_col = 0)

    # Remove index column name to make it compliant with the anndata format
    obs.index.name = None

    var = pd.read_csv(path+'Velocyto/raw/features.tsv', sep='\t',
                                    names = ('gene_ids', 'feature_types'), index_col = 1)
  
    # Build AnnData object to be used with ScanPy and ScVelo
    adata = anndata.AnnData(X = X, obs = obs, var = var,
                                                 layers = {'spliced': spliced, 'unspliced': unspliced, 'ambiguous': ambiguous})
    adata.var_names_make_unique()

    # Subset Cells based on STAR filtering
    selected_barcodes = pd.read_csv(path+'Gene/filtered/barcodes.tsv', header = None)
    adata = adata[selected_barcodes[0]]

    return adata.copy()

# Importing

In [ ]:
## Function call for buildAnndataFromStarCurr. First checks whether h5ad files already exist, then read it.
## Otherwise, create h5ad from Star output.

from os.path import exists
alldata = {}

for index, row in df.iterrows():
    print(row['Sample'], row['StarSoloFolder'])
    importFile = row['Sample'] + "-3rawImport.h5ad"
    print(importFile)
    starSoloFolder = row['StarSoloFolder']
    file_exists = exists(importFile)
    
    if file_exists:
        print("file " + importFile + " already there!")
        print("reading it ...")
        alldata[row['Sample']] = sc.read_h5ad(importFile)
    else:
        print("file " + importFile + " not there!")
        print("importing STARsolo data ...")
        alldata[row['Sample']] = buildAnndataFromStarCurr(starSoloFolder)
        alldata[row['Sample']].write_h5ad(filename=importFile)

In [ ]:
from os.path import exists

for index, row in df.iterrows():
    print(row['Sample'], row['timepoint'])        
    alldata[row['Sample']].obs['sample']=row['Sample']
    alldata[row['Sample']].obs['time']=str(row['timepoint'])

# Doublet Analysis

In [ ]:
## Scrublet parameters (DEFAULT)
#scanpy.external.pp.scrublet(adata, adata_sim=None, batch_key=None, sim_doublet_ratio=2.0, expected_doublet_rate=0.05, 
#                            stdev_doublet_rate=0.02, synthetic_doublet_umi_subsampling=1.0, knn_dist_metric='euclidean', 
#                            normalize_variance=True, log_transform=False, mean_center=True, n_prin_comps=30, use_approx_neighbors=True, 
#                            get_doublet_neighbor_parents=False, n_neighbors=None, threshold=None, verbose=True, copy=False, random_state=0)

In [ ]:
for i in alldata:
    
    alldata[i].var_names_make_unique()
    scrub = scr.Scrublet(alldata[i].X)
    alldata[i].obs['doublet_scores'],alldata[i].obs['predicted_doublets'] = scrub.scrub_doublets()
    sum(alldata[i].obs['predicted_doublets'])

# Concatenate samples

If you do not have more than 120 GB of free RAM available, you need to concatenate the data in two steps. Otherwise Scanpy will crash.
When enough RAM is available please continue at the indicated step below.

In [ ]:
type(alldata)
len(alldata)
keys_list = list(alldata)
for i in range(1,9):
        print(i)
        if i==1:
            part1 = alldata[keys_list[i-1]].concatenate(alldata[keys_list[i]])
        else:
            part1 = part1.concatenate(alldata[keys_list[i]])

In [ ]:
part1.write_h5ad(os.getcwd() + "/part1_raw_concat.h5ad")

In [ ]:
del part1
gc.collect()

In [ ]:
type(alldata)
len(alldata)
keys_list = list(alldata)
for i in range(10,len(alldata)):
        print(i)
        if i==10:
            part2 = alldata[keys_list[i-1]].concatenate(alldata[keys_list[i]])
        else:
            part2 = part2.concatenate(alldata[keys_list[i]])

In [ ]:
part2.write_h5ad(os.getcwd() + "/part2_raw_concat.h5ad")

In [ ]:
part1 = sc.read_h5ad(os.getcwd() + "/part1_raw_concat.h5ad")

In [ ]:
concatenate_data = part1.concatenate(part2)

In [ ]:
# save concatenate but raw data
concatenate_data.write_h5ad(os.getcwd() + "/concatenate_data_raw.h5ad")

In [ ]:
## remove alldata object (containing individual time point anndata samples) from memory.
del part2
del part1
del alldata
gc.collect()

When enough RAM is available, you can use this concatenation procedure.

In [ ]:
type(alldata)
len(alldata)
keys_list = list(alldata)
for i in range(1,len(alldata)):
        print(i)
        if i==1:
            concatenate_data = alldata[keys_list[i-1]].concatenate(alldata[keys_list[i]])
        else:
            concatenate_data = concatenate_data.concatenate(alldata[keys_list[i]])

In [ ]:
# save concatenate but raw data
concatenate_data.write_h5ad(os.getcwd() + "/concatenate_data_raw.h5ad")

In [ ]:
## remove alldata object (containing individual time point anndata samples) from memory.
del alldata
gc.collect()

# Cell numbers

In [ ]:
print(concatenate_data.obs['sample'].value_counts())

# QC

In [ ]:
# mitochondrial genes
concatenate_data.var['mt'] = concatenate_data.var_names.str.startswith('mt-') 
# ribosomal genes
concatenate_data.var['ribo'] = concatenate_data.var_names.str.startswith(("Rps","Rpl"))
# hemoglobin genes.
concatenate_data.var['hb'] = concatenate_data.var_names.str.contains(("^Hb[^(P)]"))
sc.pp.calculate_qc_metrics(concatenate_data, qc_vars=['mt','ribo','hb'], percent_top=None, log1p=False, inplace=True)
concatenate_data.var['Ptprc'] = concatenate_data.var_names.str.contains('Ptprc')

# Filtering  bad cells and genes

In [ ]:
# Filter cells according to identified QC thresholds:
print('Total number of cells: {:d}'.format(concatenate_data.n_obs))

sc.pp.filter_cells(concatenate_data, min_counts = 1500)
print('Number of cells after min count filter: {:d}'.format(concatenate_data.n_obs))

sc.pp.filter_cells(concatenate_data, max_counts = 40000)
print('Number of cells after max count filter: {:d}'.format(concatenate_data.n_obs))

adata = concatenate_data[concatenate_data.obs['pct_counts_mt'] < 15]
print('Number of cells after MT filter: {:d}'.format(concatenate_data.n_obs))

sc.pp.filter_cells(concatenate_data, min_genes = 2000)
print('Number of cells after gene filter: {:d}'.format(concatenate_data.n_obs))



#Filter genes:
print('Total number of genes: {:d}'.format(concatenate_data.n_vars))

# Min 20 cells - filters out 0 count genes
sc.pp.filter_genes(concatenate_data, min_cells=20)
print('Number of genes after cell filter: {:d}'.format(concatenate_data.n_vars))

In [ ]:
## Latching concatenated and quality controlled data

fileName = expName + "_concatenated_QCed.h5ad"

concatenate_data.write_h5ad(os.getcwd() + "/" + fileName)

In [ ]:
concatenate_data = sc.read_h5ad("/root/host_home/Infection_timecourse_concatenated_QCed.h5ad")

# Cell numbers after QC

In [ ]:
print(concatenate_data.obs['sample'].value_counts())


# Normalize, Cluster and Integrate

In [ ]:
sc.pp.filter_genes(concatenate_data, min_cells=20)
concatenate_data.raw = concatenate_data
# normalize to depth 10 000
sc.pp.normalize_per_cell(concatenate_data, counts_per_cell_after=1e4)

concatenate_data

In [ ]:
# logarithmize
sc.pp.log1p(concatenate_data)

In [ ]:
sc.pp.highly_variable_genes(concatenate_data, flavor='cell_ranger', n_top_genes=4000)
print('\n','Number of highly variable genes: {:d}'.format(np.sum(concatenate_data.var['highly_variable'])))

In [ ]:
sc.pl.highly_variable_genes(concatenate_data)

In [ ]:
## Integration using Harmony and subsequent embedding and UMAP calculation

integrationMethod = "harmony"
import scanpy.external as sce
# Calculate the visualizations
sc.pp.pca(concatenate_data, n_comps=50, use_highly_variable=True, svd_solver='arpack')
sce.pp.harmony_integrate(concatenate_data, key='sample')
sc.pp.neighbors(concatenate_data,use_rep = 'X_pca_harmony')
sc.tl.umap(concatenate_data)

In [ ]:
# save integrated h5ad file

fileName = expName + "_integrated.h5ad"

concatenate_data.write_h5ad(os.getcwd() + "/" + fileName)

In [ ]:
# Calculate leiden clustering with various resolutions

sc.tl.leiden(concatenate_data, key_added = "leiden_1.0") # default resolution in 1.0
sc.tl.leiden(concatenate_data, resolution = 1.4, key_added = "leiden_1.4")
sc.tl.leiden(concatenate_data, resolution = 0.6, key_added = "leiden_0.6")
sc.tl.leiden(concatenate_data, resolution = 0.4, key_added = "leiden_0.4")


In [ ]:
# set plot size parameters.
# please change according to your needs.
# This will also change the plot size of the plots you save!

plt.rcParams['figure.figsize']=(8,8)

In [ ]:
sc.pl.umap(concatenate_data, color=['leiden_1.0','leiden_0.6'], size=8)
sc.pl.umap(concatenate_data, color=['leiden_0.4','leiden_1.4'], size=8)

In [ ]:
sc.pl.umap(concatenate_data, color=['sample','time'], size=8)
sc.pl.umap(concatenate_data, color=['n_counts','leiden_1.0'], size=8)

In [ ]:
sc.pl.umap(concatenate_data, color=['doublet_score'], size=8) 
sc.pl.umap(concatenate_data, color=['predicted_doublet_str'], size=2) 

In [ ]:
plt.rcParams['figure.figsize']=(8,8)

In [ ]:
sc.pl.umap(concatenate_data, color=['predicted_doublet_str'], size=8) 

# Marker Genes

In [ ]:
#from openpyxl import Workbook
BaseDirectory = os.getcwd()

mycluster = "Manual_annotation"

# sc.pl.violin(concatenate_data, ['Acta2', 'Sftpb','Sftpc'], groupby=mycluster)
# sc.pl.violin(concatenate_data, ['Apoe', 'C1qa','Fn1'], groupby=mycluster)
# sc.pl.violin(concatenate_data, ['S100a8', 'Csf3r','Lrg1'], groupby=mycluster)
# sc.pl.violin(concatenate_data, ['Cpa3', 'Gata2','Ms4a2'], groupby=mycluster)

# sc.pl.violin(concatenate_data, ['Cd247', 'Cd96','Nkg7'], groupby=mycluster)
# sc.pl.violin(concatenate_data, ['Prf1', 'Cd3d','Cd28'], groupby=mycluster)

# sc.pl.violin(concatenate_data, ['Cd79a', 'Bank1','Tcf4'], groupby=mycluster)
# sc.pl.violin(concatenate_data, ['Tspan13', 'Pacsin1','Nucb2'], groupby=mycluster)


#markers = ['Acta2', 'Sftpb','Sftpc','Apoe', 'C1qa','Fn1','S100a8', 
#           'Csf3r','Lrg1','Cpa3', 'Gata2','Ms4a2','Cd247', 'Cd96','Nkg7','Prf1', 
#           'Cd3d','Cd28','Cd79a', 'Bank1','Tcf4','Tspan13', 'Pacsin1','Nucb2']
#sc.pl.dotplot(concatenate_data,markers, groupby=mycluster,dendrogram=True)

sc.tl.rank_genes_groups(concatenate_data,groupby=mycluster)
print(pd.DataFrame(concatenate_data.uns['rank_genes_groups']['names']).head(10))
print("")



sampleName = expName
method = 't-test' #t-test, wilcoxon, or logreg
cluster_method = mycluster
n_genes=1000 #set to adata.var.shape[0] to include all highly variable genes

sc.tl.rank_genes_groups(concatenate_data, cluster_method, use_raw=True, pts=True, n_genes=n_genes, method=method)
#sc.pl.rank_genes_groups(a, groupby=cluster_method, use_raw=False, n_genes=25, sharey=False, save='_' + str(sampleName) + '_' + method + '_' +  cluster_method + '_clusters')
pd.DataFrame(concatenate_data.uns['rank_genes_groups']['names']).head(n_genes) #ALTER NUMBER TO SPECIFIC NUMBER OF GENES TO LIST
table = pd.DataFrame(concatenate_data.uns['rank_genes_groups']['names']).head(n_genes) #ALTER NUMBER TO SPECIFIC NUMBER OF GENES TO LIST
##### table.to_excel(os.path.join(BaseDirectory, sampleName + '_' + method + '_' + cluster_method + '_cluster_table_' + str(n_genes) + 'genes.xlsx'), engine='openpyxl')
#make table with p-values included
result = concatenate_data.uns['rank_genes_groups']
groups = result['names'].dtype.names
if method != 'logreg':
    pval_table = pd.DataFrame(
            {group + '_' + key[:1]: result[key][group]
            for group in groups for key in ['names', 'pvals_adj','logfoldchanges']}).head(n_genes)
 ####   pval_table.to_excel(os.path.join(BaseDirectory, sampleName + '_' + method + '_pval_table_' + cluster_method + '_clusters_' + str(n_genes) + 'genes_filtered_corrected.xlsx'), engine='openpyxl')
elif method=='logreg':
        pval_table = pd.DataFrame(
            {group + '_' + key[:2]: result[key][group]
            for group in groups for key in ['names', 'scores']}).head(n_genes)
 ####       pval_table.to_excel(os.path.join(BaseDirectory, sampleName + '_' + method + '_coefs_' + cluster_method + '_clusters_' + str(n_genes) + 'genes_filtered_corrected.xlsx'), engine='openpyxl')

In [ ]:
pd.DataFrame(concatenate_data.uns['rank_genes_groups']['names']).head(n_genes) 

In [ ]:
# Dotplot of marker genes

In [ ]:
marker_genes = ['Itgax', 'Siglecf', 'Ear2', 'Apoe', 'Ccr5', 'Mertk', 'Fcgr1', 'Itgam', 'Cx3cr1', 'Ccr2' ,'Ly6c2' ,'Mcm3', 'Mcm4', 'Mcm5', 'Mcm6', 'Mki67', 'H2-Ab1', 'H2-Eb1', 'Ccr7', 'Cd3e', 'Cd8a', 'Cd3d', 'Gzmb', 'Gzma']

In [ ]:
sc.pl.dotplot(concatenate_data, marker_genes, groupby="Manual_annotation",dendrogram=True, log=True)

## ANNOTATION

In [ ]:
mycluster = 'leiden_1.0' # set the clustering which shall be annotated!
concatenate_data.obs['leiden_anno'] = concatenate_data.obs[mycluster]
concatenate_data.obs
res = pd.DataFrame(columns=concatenate_data.var_names, index=concatenate_data.obs['leiden_anno'].cat.categories)                                                                                                 

for clust in concatenate_data.obs.leiden_anno.cat.categories: 
    res.loc[clust] = concatenate_data[concatenate_data.obs['leiden_anno'].isin([clust]),:].X.mean(0)

In [ ]:
# This .csv file is the input for the R script below

res.transpose().to_csv("Infection_timecourse_mean_gene_expression.csv")

In [ ]:
# Automatic celltype annotation using scMCA in R

## ## ## ## R CODE
## ##
library(scMCA)
library(openxlsx)

read_counts <- read.table("/path/to/mean_reads.csv", sep="," )
rownames(read_counts) <- read_counts[,1]
read_counts <- (read_counts[,-1])
read_counts <- read_counts[-1,]

mca_result_read_counts <- scMCA(scdata = read_counts, numbers_plot = 3)

out_frame <- data.frame(cluster=gsub("X","",names(mca_result_read_counts[[3]])),type=mca_result_read_counts[[3]])
write.csv(out_frame, file="/out/file/path/annotation.csv",row.names = F) # PLEASE CHANGE PATH ACCORDINGLY!


In [ ]:
df = pd.read_csv("/out/file/path/annotation.csv",sep=",") # PLEASE CHANGE PATH ACCORDINGLY!
print(df)
cellTypes = df.to_dict()['type']
cellTypes

In [ ]:
type(concatenate_data.obs['leiden_anno'])
leiden = (concatenate_data.obs['leiden_anno'].to_numpy(dtype=None))
leiden + "_" + [cellTypes[x] for x in leiden.astype(int)] 

In [ ]:
mcacluster = mycluster+'_MCA'
concatenate_data.obs[mcacluster] = leiden + "_" + [cellTypes[x] for x in leiden.astype(int)] 

# Manual annotation!

In [ ]:
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4']


concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='0', value='1')
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='2', value='1')
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='3', value='1')
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='5', value='1')
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='6', value='1')
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='8', value='1') 
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='9', value='1') 
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='10', value='1') 
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='11', value='1')
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='15', value='1')
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='16', value='1')
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='17', value='1')
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='12', value='2')
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='13', value='3')
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='18', value='X')
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='4', value='5')
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='20', value='5')
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='7', value='6')
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='14', value='7')
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='19', value='8')
concatenate_data.obs['leiden_1.4_new'] = concatenate_data.obs['leiden_1.4_new'].replace(to_replace='X', value='4')

In [ ]:
# Reorder categories
concatenate_data.obs['leiden_1.4_new'].cat.reorder_categories(['1', '2', '3', '4', '5', '6', '7', '8'], inplace=True)

In [ ]:
concatenate_data.obs['Manual_annotation'] = concatenate_data.obs['leiden_1.4_new']

concatenate_data.obs['Manual_annotation'] = concatenate_data.obs['Manual_annotation'].replace(to_replace='1', value='TR-AM')
concatenate_data.obs['Manual_annotation'] = concatenate_data.obs['Manual_annotation'].replace(to_replace='2', value='Cycling_TR-AM_1')
concatenate_data.obs['Manual_annotation'] = concatenate_data.obs['Manual_annotation'].replace(to_replace='3', value='Cycling_TR-AM_2')
concatenate_data.obs['Manual_annotation'] = concatenate_data.obs['Manual_annotation'].replace(to_replace='4', value='Cmpk2-high_TR_AM')
concatenate_data.obs['Manual_annotation'] = concatenate_data.obs['Manual_annotation'].replace(to_replace='5', value='BMDM1')
concatenate_data.obs['Manual_annotation'] = concatenate_data.obs['Manual_annotation'].replace(to_replace='6', value='BMDM2')
concatenate_data.obs['Manual_annotation'] = concatenate_data.obs['Manual_annotation'].replace(to_replace='7', value='DC')
concatenate_data.obs['Manual_annotation'] = concatenate_data.obs['Manual_annotation'].replace(to_replace='8', value='T_Cells')

In [ ]:
plt.rcParams['figure.figsize']=(8,8)
sc.set_figure_params(dpi=300)

In [ ]:
# Manually set color palette
mpl = ["steelblue","red","purple","pink","darkorange","green","sienna","grey"]

In [ ]:
sc.pl.umap(concatenate_data, color=['Manual_annotation'], palette=mpl,frameon=False, size=8)

In [ ]:
# save annotated h5ad file

fileName = expName + "_annotated.h5ad"

concatenate_data.write_h5ad(os.getcwd() + "/" + fileName)

# Violin Plots

In [ ]:
sub_0 = concatenate_data[concatenate_data.obs["time"] == "d_00"]
sub_7 = concatenate_data[concatenate_data.obs["time"] == "d_07"]
sub_14 = concatenate_data[concatenate_data.obs["time"] == "d_14"]
sub_21 = concatenate_data[concatenate_data.obs["time"] == "d_21"]
sub_35 = concatenate_data[concatenate_data.obs["time"] == "d_35"]

In [ ]:
sc.pl.violin(sub_35, keys = ['Plet1'], groupby= "Manual_annotation", use_raw=False, rotation=90)

# Trackplot

In [ ]:
## Trackplot
mg = pd.read_csv("tracklist_genes.csv",sep=",")
mg['genes'].values.tolist()
sc.pl.tracksplot(concatenate_data, var_names=mg['genes'].values.tolist(),groupby="Manual_annotation",save="trackplot_version1.png",dendrogram=False,raw=False,palette=mpl)
sc.pl.tracksplot(concatenate_data, var_names=mg['genes'].values.tolist(),groupby="Manual_annotation",save="trackplot_version2.png",dendrogram=True,raw=False,palette=mpl)

## VELOCITY

In [ ]:
import scvelo as scv
from IPython.display import display, Markdown, Latex

In [ ]:
concatenate_data = sc.read_h5ad("/root/host_home/Infection_timecourse_leiden_clustered_mareks_docker.h5ad")

In [ ]:
for i in concatenate_data.obs['time'].unique():
    asub = concatenate_data[concatenate_data.obs['time']==str(i)].copy()
    asub.obs['clusters'] = asub.obs['Manual_annotation']
    asub = asub[-asub.obs['Manual_annotation'].isin(['7','8'])]
    print(asub)
    print('Timepoint:' + i)
    display(Markdown('### time point: {}'.format(i)))
    scv.pl.proportions(asub)
    scv.pp.filter_genes(asub, min_shared_counts=100)
    scv.pp.normalize_per_cell(asub)
    scv.pp.filter_genes_dispersion(asub, n_top_genes=500)
    #scv.pp.log1p(a)
    scv.pp.filter_and_normalize(asub, min_shared_counts=20, n_top_genes=500)
    scv.pp.moments(asub, n_pcs=30, n_neighbors=30)
    scv.tl.velocity(asub)
    scv.tl.velocity_graph(asub)
    outfilename = "velocity_timepoint_" + i +"_colored.png"
    scv.pl.velocity_embedding_stream(asub, basis='umap',save=outfilename,palette=mpl)


    mycluster="Manual_annotation"
    scv.tl.rank_velocity_genes(asub, groupby=mycluster, min_corr=.3)
    df = scv.DataFrame(asub.uns['rank_velocity_genes']['names'])

    scv.tl.velocity_pseudotime(asub)
    scv.pl.scatter(asub, color='velocity_pseudotime', cmap='gnuplot')


    # this is needed due to a current bug - bugfix is coming soon.
    asub.uns['neighbors']['distances'] = asub.obsp['distances']
    asub.uns['neighbors']['connectivities'] = asub.obsp['connectivities']

    scv.tl.paga(asub, groups=mycluster)
    df = scv.get_df(asub, 'paga/transitions_confidence', precision=2).T
    df.style.background_gradient(cmap='Blues').format('{:.2g}')

    outfilename = "paga_timepoint_" + i +"_colored.png"

    scv.pl.paga(asub, basis='umap', size=50, alpha=.1,min_edge_width=2, node_size_scale=1.5,save=outfilename,palette=mpl)